# Load the models

In [2]:
import cobra
from cobra import Reaction, Metabolite

In [3]:
# import the models
from cobra.io import load_json_model
ecoli = load_json_model("ecoli.json")
pf = load_json_model("pf.json")
# print(ecoli.optimize())
# print(pf.optimize())
    
# load peripheral packages
import warnings
warnings.filterwarnings(action='once')
from pandas import set_option
set_option("display.max_rows", None)
from time import process_time

# Load the conditions

In [4]:
# Define the standard graphs from each simulation
graphs_list = [
    {
        'trial':'G48',
        "phenotype": '*',
        'content': 'biomass',
        'experimental_data': False,
        # "parsed":True
    },
    {
        'trial':'G48',
        'content': "conc",
    },
    {
        'trial':'G48',
        "phenotype": '*',
        'content': 'biomass',
        "parsed": True
    },
    {
        'trial':'G48',
        'content': 'total_biomass',
        'experimental_data': True
    }
]

def update_graphs(trial):
    graphs = graphs_list.copy()
    for graph in graphs:
        graph["trial"] = trial
    return graphs

In [5]:
species_abundances = {
    1:{"ecoli":0, "pf":1},
    2:{"ecoli":1, "pf":50},
    3:{"ecoli":1, "pf":20},
    4:{"ecoli":1, "pf":10},
    5:{"ecoli":1, "pf":3},
    6:{"ecoli":1, "pf":1},
    7:{"ecoli":3, "pf":1},
    8:{"ecoli":10, "pf":1},
    9:{"ecoli":20, "pf":1},
    10:{"ecoli":1, "pf":0},
    11:{"ecoli":0, "pf":0}
}
carbon_conc_series = {'rows': {
    'cpd00136': {'B':0, 'C': 0, 'D': 1, 'E': 1, 'F': 4, 'G': 4},
    'cpd00179': {'B':5, 'C': 5, 'D':5, 'E': 5, 'F': 5, 'G': 5},
    'cpd00100': {'B':5, 'C': 5, 'D':5, 'E': 5, 'F': 5, 'G': 5} # glycerol
}}
species_identities_rows = {
    1:{"ecoli":"mRuby"},
    2:{"ecoli":"ACS"},
    3:{"ecoli":"mRuby"},
    4:{"ecoli":"ACS"},
    5:{"ecoli":"mRuby"},
    6:{"ecoli":"ACS"}
}

# Simulate

In [6]:
# checking if the metabolite of interest is in the E.coli model

exists = "false"
for met in pf.metabolites:
    if met.id == "cpd00010_c0":
        exists = "true"
print(exists)

true


## Parameterize the data

### A bioproduction phenotype

In [7]:
# checking if the exchange reaction is in P.fluorescens model

exists = "false"
for ex in pf.exchanges:
    if ex.id == "EX_cpd00480_c0":
        exists = "true"
print(exists)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


false


In [8]:
# # create requisite exchange metabolite
# #cyto_met = ecoli.metabolites.get_by_id("cpd00150_c0")
# cyto_met = pf.metabolites.get_by_id("cpd00150_c0")
# ex_met = cyto_met.copy()
# ex_met.id = "cpd00150_c0"
# ex_met.compartment = "e0"

# # add metabolite to each model
# #ecoli.add_metabolites([ex_met])
# pf.add_metabolites([ex_met])

# # add exchange reactions
# #ecoli.add_boundary(ex_met, type="exchange")
# pf.add_boundary(ex_met, type="exchange")

In [9]:
# defining corresponding transport reaction
# cpd00154_tr = Reaction("cpd00154_tr")
# cpd00154_tr.name = "cpd00154 Transport"
# cpd00154_tr.add_metabolites({
#     ex_met: -1, cyto_met: 1
# })

In [10]:
# finding reactions containing the metabolite of interest in our model
for rxn in pf.metabolites.cpd00100_c0.reactions:
    print(rxn)

rxn29769_c0: cpd00001_c0 + cpd00002_c0 + cpd00100_e0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + cpd00100_c0
rxn10336_c0: 2 cpd15540_c0 --> cpd00100_c0 + cpd15793_c0
rxn08669_c0: cpd00001_c0 + cpd02090_c0 --> cpd00067_c0 + cpd00080_c0 + cpd00100_c0
rxn05581_c0: cpd00100_c0 <=> cpd00100_e0
rxn00615_c0: cpd00002_c0 + cpd00100_c0 --> cpd00008_c0 + cpd00067_c0 + cpd00080_c0
rxn00763_c0: cpd00003_c0 + cpd00100_c0 <=> cpd00004_c0 + cpd00067_c0 + cpd00448_c0


In [11]:
# adding metabolites for rxn02469
cpd00024_c0 = Metabolite(
    'cpd00024_c0',
    formula = 'C5H4O5',
    name = '2-Oxoglutarate',
    compartment = 'c')
cpd02374_c0 = Metabolite(
    'cpd02374_c0',
    formula = 'C9H21N2O2',
    name = 'N6,N6,N6-Trimethyl-L-lysine',
    compartment = 'c')
cpd00923_c0 = Metabolite(
    'cpd00923_c0',
    formula = 'C9H21N2O3',
    name = '3-Hydroxy-N6,N6,N6-trimethyl-L-lysine',
    compartment = 'c')

pf.add_metabolites([cpd00024_c0, cpd02374_c0, cpd00923_c0])

rxn02469 = Reaction(id = "rxn02469")
rxn02469.add_metabolites({
    pf.metabolites.cpd00007_c0: -1, 
    pf.metabolites.cpd00024_c0: -1, 
    pf.metabolites.cpd02374_c0: -1, 
    pf.metabolites.cpd00011_c0: 1,
    pf.metabolites.cpd00036_c0: 1,
    pf.metabolites.cpd00923_c0: 1
})
pf.add_reactions([rxn02469])

In [22]:
# adding metabolites for rxn01727
cpd00024_c0 = Metabolite(
    'cpd00024_c0',
    formula = 'C5H4O5',
    name = '2-Oxoglutarate',
    compartment = 'c')
cpd00870_c0 = Metabolite(
    'cpd00870_c0',
    formula = 'C7H15NO2',
    name = 'gamma-butyrobetaine',
    compartment = 'c')
cpd00266_c0 = Metabolite(
    'cpd00266_c0',
    formula = 'C7H15NO3',
    name = 'Carnitine',
    compartment = 'c')
cpd00266_e0 = Metabolite(
    'cpd00266_e0',
    formula = 'C7H15NO3',
    name = 'Carnitine',
    compartment = 'e0')

pf.add_metabolites([cpd00024_c0, cpd00870_c0, cpd00266_c0, cpd00266_e0])
pf.add_boundary(cpd00266_e0, type = "exchange")

# adding rxn01727
rxn01727 = Reaction(id = "rxn01727")

rxn01727.add_metabolites({
    pf.metabolites.cpd00007_c0: -1, 
    pf.metabolites.cpd00024_c0: -1, 
    pf.metabolites.cpd00870_c0: -1, 
    pf.metabolites.cpd00011_c0: 1,
    pf.metabolites.cpd00036_c0: 1,
    pf.metabolites.cpd00266_c0: 1
})
pf.add_reactions([rxn01727])

ValueError: Boundary reaction 'EX_cpd00266_e0' already exists.

In [13]:
# adding mets for rxn00032
cpd00098_c0 = Metabolite(
    'cpd00098_c0',
    formula = 'C5H14NO',
    name = 'Choline',
    compartment = 'c')
cpd00447_c0 = Metabolite(
    'cpd00447_c0',
    formula = 'C5H12NO',
    name = 'Betaine aldehyde',
    compartment = 'c')
pf.add_metabolites([cpd00098_c0, cpd00447_c0])

# adding rxn00032
rxn00032 = Reaction(id = "rxn00032")
rxn00032.add_metabolites({
    pf.metabolites.cpd00007_c0: -1,
    pf.metabolites.cpd00098_c0: -2,
    pf.metabolites.cpd00001_c0: 2,
    pf.metabolites.cpd00447_c0: 2
})
pf.add_reactions([rxn00032])

In [14]:
# adding mets for rxn01867
cpd00540_c0 = Metabolite(
    'cpd00540_c0',
    formula = 'C5H11NO2',
    name = "Betaine",
    compartment = 'c')
pf.add_metabolites(cpd00540_c0)

# adding rxn01867
rxn01867 = Reaction(id = "rxn01867")
rxn01867.add_metabolites({
    pf.metabolites.cpd00001_c0: -1,
    pf.metabolites.cpd00003_c0: -1,
    pf.metabolites.cpd00447_c0: -1,
    pf.metabolites.cpd00004_c0: 1,
    pf.metabolites.cpd00067_c0: 2,
    pf.metabolites.cpd00540_c0: 1
})
pf.add_reactions([rxn01867])

In [15]:
# adding RXN-22878 (BioCyc id)
cpd00870_c0 = Metabolite(
    'cpd00870_c0',
    formula = 'C7H15NO2',
    name = 'gamma-butyrobetaine',
    compartment = 'c')
cpd15412_c0 = Metabolite(
    'cpd15412_c0',
    formula = 'C28H46N8O17P3S',
    name = 'gamma-butyrobetainyl-CoA',
    compartment = 'c')
pf.add_metabolites([cpd00870_c0, cpd15412_c0])

# adding RXN-22878
RXN_22878 = Reaction(id = "RXN-22878")
RXN_22878.add_metabolites({
    pf.metabolites.cpd00870_c0: -1,
    pf.metabolites.cpd00022_c0: -1,
    pf.metabolites.cpd15412_c0: 1,
    pf.metabolites.cpd00029_c0: 1
})
pf.add_reactions([RXN_22878])

In [16]:
# adding mets for rxn19329
cpd22290_c0 = Metabolite(
    'cpd22290_c0',
    name = 'Acceptor',
    compartment = 'c')
cpd15441_c0 = Metabolite(
    'cpd15441_c0',
    formula = 'C28H44N8O17P3S',
    name = 'crotonobetainyl-CoA',
    compartment = 'c')
cpd26978_c0 = Metabolite(
    'cpd26978_c0',
    name = 'Donor-H2',
    compartment = 'c')
pf.add_metabolites([cpd22290_c0, cpd15441_c0, cpd26978_c0])

# adding rxn19329
rxn19329 = Reaction(id = "rxn19329")
rxn19329.add_metabolites({
    pf.metabolites.cpd15412_c0: -1,
    pf.metabolites.cpd22290_c0: -1,
    pf.metabolites.cpd26978_c0: 1,
    pf.metabolites.cpd15441_c0: 1
})
pf.add_reactions([rxn19329])

In [17]:
# adding mets for rxn08258
cpd15440_c0 = Metabolite(
    'cpd15440_c0',
    formula = 'C28H46N8O18P3S',
    name = 'Carnitinyl-CoA',
    compartment = 'c')
pf.add_metabolites(cpd15440_c0)

# adding rxn08258
rxn08258 = Reaction(id = "rxn08258")
rxn08258.add_metabolites({
    pf.metabolites.cpd00001_c0: -1,
    pf.metabolites.cpd15441_c0: -1,
    pf.metabolites.cpd15440_c0: 1
})
pf.add_reactions([rxn08258])

In [18]:
# adding RXN-22886 (BioCyc id)
# note that we do not have to define mets as they had been previously defined to previous reactions and already exist in the model
RXN_22886 = Reaction(id = "RXN-22886")
RXN_22886.add_metabolites({
    pf.metabolites.cpd15440_c0: -1,
    pf.metabolites.cpd00001_c0: -1,
    pf.metabolites.cpd00010_c0: 1,
    pf.metabolites.cpd00266_c0: 1
})
pf.add_reactions([RXN_22886])

In [19]:
solution = pf.optimize()
print(solution)

pf.summary()

<Solution 0.494 at 0x13140a1c0>


Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_e0,EX_cpd00007_e0,10,0,0.00%
cpd00009_e0,EX_cpd00009_e0,0.3765,0,0.00%
cpd00013_e0,EX_cpd00013_e0,4.252,0,0.00%
cpd00029_e0,EX_cpd00029_e0,15.13,2,100.00%
cpd00030_e0,EX_cpd00030_e0,0.001385,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.001385,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.1118,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.001385,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.001385,0,0.00%
cpd00067_e0,EX_cpd00067_e0,11.91,0,0.00%


In [23]:
print(pf.reactions)
#print(pf.metabolites)

[<Reaction rxn02201_c0 at 0x131669490>, <Reaction rxn00351_c0 at 0x131669be0>, <Reaction rxn07431_c0 at 0x131669b50>, <Reaction rxn00836_c0 at 0x131669b20>, <Reaction rxn00423_c0 at 0x131676610>, <Reaction rxn00364_c0 at 0x131676f70>, <Reaction rxn05561_c0 at 0x131676ee0>, <Reaction rxn03408_c0 at 0x13167af10>, <Reaction rxn02177_c0 at 0x13167aeb0>, <Reaction rxn05250_c0 at 0x13167e5e0>, <Reaction rxn00646_c0 at 0x13167ed90>, <Reaction rxn00935_c0 at 0x13167ed30>, <Reaction rxn01673_c0 at 0x13167ed00>, <Reaction rxn00247_c0 at 0x131683970>, <Reaction rxn05625_c0 at 0x131683d60>, <Reaction rxn02360_c0 at 0x131686d30>, <Reaction rxn02342_c0 at 0x131686cd0>, <Reaction rxn05156_c0 at 0x131686ca0>, <Reaction rxn01452_c0 at 0x13168acd0>, <Reaction rxn05667_c0 at 0x13168ac70>, <Reaction rxn02000_c0 at 0x13168ab20>, <Reaction rxn03534_c0 at 0x13168f610>, <Reaction rxn00322_c0 at 0x13168f910>, <Reaction rxn00654_c0 at 0x13168fee0>, <Reaction rxn05457_c0 at 0x13168f580>, <Reaction rxn00107_c0 at

In [24]:
from modelseedpy import MSCommPhitting

test_com = MSCommPhitting('/Users/ashleyeng/Documents/Python/ModelSEEDDatabase',
                           community_members = {
                                ecoli: {
                                    'name': 'ecoli', 
                                    'phenotypes': {
                                        'Maltose': {"consumed":["cpd00179"],
                                               "excreted":["cpd00029"]}, 
                                        'Acetate': {"consumed":["cpd00029"]},
                                        #'Xylose': {"consumed":["cpd00154"],
                                               # "excreted":["cpd00363"]}
                                     }
                                },
                                pf: {
                                    'name': 'pf',
                                    'phenotypes': {
                                        'Acetate': {"consumed":["cpd00029"]},
                                        'Glycerol': {"consumed":["cpd00100"],
                                               "excreted":["cpd00266"]}
                                    }
                                }
                            },
                            data_paths = {'path':"data/Jeffs_data/PF-EC 4-29-22 ratios and 4HB changes.xlsx", 'Raw OD(590)':'OD', 'mNeonGreen':'pf', 'mRuby':'ecoli'},
                            species_abundances = species_abundances,
                            carbon_conc = carbon_conc_series,
                            ignore_trials = {'rows': ['C', 'D', 'E', 'F', 'G'], 'columns': [1,2,3,4,5,6,8,9,10,11,12]},
                            species_identities_rows = species_identities_rows)

test_com.fit(parameters = {'cvmin': .1, 'bcv': 0.0001, "stationary": 5e-3},
             mets_to_track = ["cpd00100","cpd00266"],
             rel_final_conc = {"cpd00100":0.1},
             zero_start = ["cpd00266"],
             #abs_final_conc = {"cpd00029":0}, # metabolomics data
             graphs=update_graphs("G6"),
             export_zip_name = 'glycerol.zip',
             export_lp = "glycerol.lp",
             primals_export_path="test_primals_simple.json")


 GCF_000005845.2.RAST.mdl
EX_cpd00179_e0
EX_cpd00029_e0

 GCF_001612705.1.RAST.mdl
EX_cpd00029_e0
EX_cpd00100_e0
The ['B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B9', 'B10', 'B11', 'B12', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12'] trials were dropped from the OD measurements per the ignore_trials parameter.
The [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166] timesteps (with [1.12550559 1.12537038 1.12

,ecoli_Maltose,ecoli_Acetate,pf_Acetate,pf_Glycerol
rxn,,,,
EX_cpd00001_e0,0.000000,4.544367e+00,4.533296e+00,4.487274
EX_cpd00007_e0,-22.000910,-2.035609e+00,-2.025521e+00,-1.214762
EX_cpd00009_e0,-0.076254,-7.625410e-02,-7.625410e-02,-0.076254
EX_cpd00011_e0,28.376856,2.388151e+00,2.377641e+00,0.802145
EX_cpd00013_e0,-0.861167,-8.611666e-01,-8.611666e-01,-0.861167
EX_cpd00029_e0,100.000000,-3.069851e+00,-3.063614e+00,0.018343
EX_cpd00030_e0,-0.000281,-2.806159e-04,-2.806159e-04,-0.000281
EX_cpd00034_e0,-0.000281,-2.806159e-04,-2.806159e-04,-0.000281
EX_cpd00048_e0,-0.240125,-2.263972e-02,-2.263972e-02,-0.022640


KeyError: 'EX_cpd00266_e0'